# Setup

msticpy is a set of Python tools intended to be used for security investigations and hunting. Many of the tools originated as code Jupyter notebooks written to solve a problem as part of a security investigation. Some of the tools are only useful in notebooks (e.g. the widgets and visualizations) but many others can be used from the Python commandline or imported into your code.

MSTICpy includes a feature called nbinit that handles the process of installing and importing modules into a notebook environment. This was developed to allow for a clearer starting cell in notebooks and to avoid users being presented with a very large cell block at the top of a notebook.
By passing the notebook namespace to init_notebook() this function handles the job of installing and importing core MSTICpy packages along with any others that might be needed by a notebook.

Find more details for the instalation from [MSTICPY](https://msticpy.readthedocs.io/en/latest/getting_started/Installing.html#).



In [ ]:
from msticpy.nbtools import nbinit
extra_imports = [
    "msticpy.vis.timeseries, display_timeseries_anomolies",
    "msticpy.analysis.timeseries, timeseries_anomalies_stl",
    "datetime, datetime",
    "msticpy.vis.nbdisplay, draw_alert_entity_graph",
    "msticpy.context.ip_utils, convert_to_ip_entities",
    "msticpy.vis.ti_browser, browse_results",
    "IPython.display, Image",
    "msticpy.context.ip_utils, get_whois_info",
    "msticpy.context.ip_utils, get_ip_type"
    
]

nbinit.init_notebook(
    namespace=globals(),
    additional_packages=["pyvis"],
    extra_imports=extra_imports,
);

from msticpy.context import TILookup
ti = TILookup()

In [ ]:
# Initalize and connect to Azure Sentinel using details from our config file.
qry_prov = QueryProvider('LogAnalytics')
wkspace = WorkspaceConfig()
qry_prov.connect(wkspace.code_connect_str)

# Threat Intelligence enrichments 
Finding a Threat Intelligence Indicators for tracking the IP address involements in any cloud operations, Here you can utilise a your owen threat intelligence feeds or API calls, we are using a Tredinals TI based feeds from sentinel loag analytics tables


In [ ]:
query = f"""ThreatIntelligenceIndicator
|where ConfidenceScore >= 80
|extend NetworkIOCType= iff(isnotempty(DomainName),"Domain","NetworkIP")
|extend IndicatorType= coalesce(FileHashType,NetworkIOCType), TIIoc= coalesce(NetworkSourceIP,FileHashValue,DomainName)
|distinct ThreatType,IndicatorType,TIIoc,ActivityGroupNames
|limit 1000"""

ThreatintelligenceIOCs = qry_prov.exec_query(query)
ThreatintelligenceIOCs = ThreatintelligenceIOCs.to_dataframe()
display(ThreatintelligenceIOCs)

# Explore the TI indicatores in AWS, GCP and Azure

Below cell executions will show you cloud resource access via suspicious IP addresses, where we can find possible compromised users or cloud resources across AWS, GCP, or Azure clouds Env.

In [ ]:
# AWS CloudTrail logs are a great source of information for monitoring and auditing AWS account activity. 

ThreatintelligenceIOCs_IP = list(ThreatintelligenceIOCs[ThreatintelligenceIOCs['IndicatorType'] == 'NetworkIP']['TIIoc'])
ThreatintelligenceIOCs_Domain = list(ThreatintelligenceIOCs[ThreatintelligenceIOCs['IndicatorType'] == 'Domain']['TIIoc'])
ThreatintelligenceIOCs_FileHash = list(ThreatintelligenceIOCs[ThreatintelligenceIOCs['IndicatorType'] == 'FileHash']['TIIoc'])

AWSCloudTrails_IP_View = f"""AWSCloudTrail
            | where TimeGenerated > ago(30d)
        | where SourceIPAddress in ({ThreatintelligenceIOCs_IP})
        | where ActionType == "Success"
        | project
            arn
            EventName,
            EventSource,
            EventTypeName,
            RecipientAccountId,
            ResponseElements,
            SessionMfaAuthenticated,
            SourceIpAddress,
            TimeGenerated,
            UserAgent,
            UserIdentityArn,
            UserIdentityType,
            UserIdentityUserName"""
AWSCloudTrails_IP_View = qry_prov.exec_query(AWSCloudTrails_IP_View)
AWSCloudTrails_IP_View = AWSCloudTrails_IP_View.to_dataframe()
display(AWSCloudTrails_IP_View)

In [ ]:
# GCP Audit logs are a great source of information for monitoring and auditing GCP account activity.

GCPAuditLogs_IP_View = f"""GCPAuditLogs 
            | where TimeGenerated > ago(30d)
      // Extract and transform relevant GCP Audit Log attributes
      | extend
          GCPUserUPN= tostring(parse_json(AuthenticationInfo).principalEmail),
          GCPUserIp = tostring(parse_json(RequestMetadata).callerIp),
          GCPUserUA= tostring(parse_json(RequestMetadata).callerSuppliedUserAgent),
          Tags= tostring(parse_json(Request).tags),
          RequestJS = parse_json(Request)
      | where GCPuserIp in ({ThreatintelligenceIOCs_IP})
      | where GCPUserUPN !has "gserviceaccount.com"
      | extend Name = tostring(split(GCPUserUPN, "@")[0]), UPNSuffix = tostring(split(GCPUserUPN, "@")[1])
      // Select relevant attributes for further analysis
      | project
          GCPOperationTime=TimeGenerated,
          GCPUserUPN,
          GCPUserIp,
          GCPUserUA,
          Name,
          UPNSuffix"""
GCPAuditLogs_IP_View = qry_prov.exec_query(GCPAuditLogs_IP_View)
GCPAuditLogs_IP_View = GCPAuditLogs_IP_View.to_dataframe()  
display(GCPAuditLogs_IP_View)

In [ ]:
# Azure Signin logs are a great source of information for monitoring and auditing Azure account activity.

AzureSiginLogs_IP_View = f"""SigninLogs
            | where TimeGenerated > ago(30d)
            //| where AppDisplayName == "Azure Portal"
            | where isnotempty(OriginalRequestId)
            | where IPAddress in ({ThreatintelligenceIOCs_IP})
            | project 
                IPAddress, 
                UserPrincipalName, 
                TimeGenerated,
                UserAgent,
                ConditionalAccessStatus,
                OperationName,
                RiskDetail,
                AuthenticationRequirement,
                ClientAppUsed 
            // Extracting the name and UPN suffix from UserPrincipalName
            | extend
                Name = tostring(split(UserPrincipalName, "@")[0]),
                UPNSuffix = tostring(split(UserPrincipalName, "@")[1])"""
AzureSiginLogs_IP_View = qry_prov.exec_query(AzureSiginLogs_IP_View)
AzureSiginLogs_IP_View = AzureSiginLogs_IP_View.to_dataframe()
display(AzureSiginLogs_IP_View)